<a href="https://colab.research.google.com/github/gouri-22/bias-aware-income-classification/blob/main/bias_fairness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
from fairlearn.datasets import fetch_adult
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report
from fairlearn.metrics import MetricFrame, selection_rate, demographic_parity_difference
from fairlearn.postprocessing import ThresholdOptimizer
from fairlearn.reductions import ExponentiatedGradient, DemographicParity

In [4]:
!pip install fairlearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 5.7 MB/s eta 0:00:00


In [8]:
data = fetch_adult(as_frame=True)
X_raw = data.data.copy()
y = data.target

In [9]:
X_raw.replace("?", np.nan, inplace=True)
X_raw.dropna(inplace=True)
y = y[X_raw.index]

In [14]:
categorical_cols = X_raw.select_dtypes(include='object').columns
numeric_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

In [15]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_encoded = encoder.fit_transform(X_raw[categorical_cols])
X_encoded_df = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(categorical_cols))

In [16]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_raw[numeric_cols])
X_scaled_df = pd.DataFrame(X_scaled, columns=numeric_cols)

X_preprocessed = pd.concat([X_scaled_df.reset_index(drop=True), X_encoded_df.reset_index(drop=True)], axis=1)

In [18]:
# PCA
n_components = min(10, X_preprocessed.shape[1])
pca = PCA(n_components=n_components)
X_reduced = pca.fit_transform(X_preprocessed)

In [25]:
y = y.map({'>50K': 1, '<=50K': 0})

In [26]:
# Train/test split
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(X_reduced, y, X_raw.index, test_size=0.3, stratify=y, random_state=42)
sensitive_train = X_raw.loc[idx_train, 'sex']
sensitive_test = X_raw.loc[idx_test, 'sex']

In [27]:
# Train baseline model
model = LogisticRegression(max_iter=500)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [28]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.8063683938969558
              precision    recall  f1-score   support

           0       0.82      0.95      0.88     10205
           1       0.70      0.38      0.49      3362

    accuracy                           0.81     13567
   macro avg       0.76      0.66      0.69     13567
weighted avg       0.79      0.81      0.78     13567



In [29]:
metric_frame = MetricFrame(
    metrics={'accuracy': accuracy_score, 'selection_rate': selection_rate},
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=sensitive_test
)
print(metric_frame.by_group)

        accuracy  selection_rate
sex                             
Female  0.884798          0.0829
Male    0.768381          0.1593


In [30]:
dp_gap = demographic_parity_difference(y_test, y_pred, sensitive_features=sensitive_test)
print("Demographic Parity Difference:", dp_gap)

Demographic Parity Difference: 0.07639939717439087


In [31]:
# Fairness mitigation - ThresholdOptimizer
postproc = ThresholdOptimizer(estimator=model, constraints="demographic_parity", prefit=True)
postproc.fit(X_train, y_train, sensitive_features=sensitive_train)
y_pred_post = postproc.predict(X_test, sensitive_features=sensitive_test)

In [32]:
dp_post = demographic_parity_difference(y_test, y_pred_post, sensitive_features=sensitive_test)
acc_post = accuracy_score(y_test, y_pred_post)
print("Post-mitigation Accuracy:", acc_post)
print("Post-mitigation Demographic Parity Difference:", dp_post)

Post-mitigation Accuracy: 0.7947224883909486
Post-mitigation Demographic Parity Difference: 0.016935489850178356


In [33]:
#Fairness mitigation-ExponentiatedGradient
expgrad = ExponentiatedGradient(LogisticRegression(), constraints=DemographicParity())
expgrad.fit(X_train, y_train, sensitive_features=sensitive_train)
y_pred_exp = expgrad.predict(X_test)

dp_exp = demographic_parity_difference(y_test, y_pred_exp, sensitive_features=sensitive_test)
acc_exp = accuracy_score(y_test, y_pred_exp)
print("Exponentiated Gradient Accuracy:", acc_exp)
print("Exponentiated Gradient Demographic Parity Difference:", dp_exp)

Exponentiated Gradient Accuracy: 0.7715043856416304
Exponentiated Gradient Demographic Parity Difference: 0.003094943056310015
